In [2]:
import pandas as pd

### Get source data

In [3]:
dhc_folder = 'U:/Source Files/Data Analytics/Data-Science/Data/DHC'
file = f'{dhc_folder}/DHC_2021-02.csv'

In [4]:
dhc = pd.read_csv(file, dtype=str)

In [9]:
dhc['Phone Number'] = dhc['Phone Number'].apply(lambda x: str(x).replace('.','').strip())

In [10]:
dhc.head()

,NPI,ME,Additional Office Locations,Phone Number,Zip Code,State,City,Address,First Name,Physician Name,Middle Name,Primary Specialty,Primary Hospital Affiliation,Last Name,Address1
0,1528265030,04201040791,NaN,7877580640,"=""00929""",PR,San Juan,Anestesiologia Rcm,Elfren,Elfren Colon,A.,Anesthesiology - Anesthesiologist,NaN,Colon,Apartado 29134
1,1447466313,49575000010,NaN,5613701363,33428,FL,Boca Raton,23123 FL-7,Rashmi,Rashmi Kar,NaN,Obstetrics/Gynecology,"Sylvester Comprehensive Cancer Center (Miami, FL)",Kar,NaN
2,1447468301,02106050565,"Baton Rouge, LA",9853450050,70403,LA,Hammond,1615 SW Railroad Ave,Annette,Annette Rhodes,M,Ophthalmology,"Cypress Pointe Hospital (Hammond, LA)",Rhodes,1615 SW Railroad Ave
3,1447468970,04113040195,NaN,4122624911,15108,PA,Coraopolis,980 Beaver Grade Rd,Susannah,Susannah Berke,NaN,Dermatology,"Heritage Valley Beaver (Beaver, PA)",Berke,NaN
4,1447472956,03840060038,"Murray, UT-Ogden, UT-Salt Lake Cty, UT",8015012600,84094,UT,Sandy,9660 1300 E,Spencer,Spencer Proctor,J,Emergency Medicine,"Riverton Hospital (Riverton, UT)",Proctor,NaN


### Load comparison population

In [6]:
data = pd.read_excel('C:/Users/glappe/Documents/validated_phones_and_polos_2021-03-23.xlsx')

In [7]:
data.head()

,ME_NO,FNAME,MNAME,LNAME,OFFICE_PHONE,ADDR1,ADDR2,ADDR_CITY,ADDR_STATE,ADDR_ZIP
0,4204880064,CARMEN,J,SIERRA LUCIANO,7872818522,NaN,400 CALLE MANUEL DOMENECH STE 311,SAN JUAN,PR,918
1,3545120088,ERIN,ELIZABETH,CASEY,6364417280,NaN,11 GARVEY PKWY,SAINT CHARLES,MO,63303
2,69007840063,OBIOHA,LOUIS,OKORO,4787466662,NaN,844 2ND ST,MACON,GA,31201
3,65501960025,MOUNIR,NaN,BELCADI,8056526729,NaN,200 HILLMONT AVE,VENTURA,CA,93003
4,802951248,GORDON,SCOTT,ABRAMS,4126411635,MAGEE-WOMENS HOSPITAL,300 HALKET ST,PITTSBURGH,PA,15213


In [12]:
data['ME_NO'] = data['ME_NO'].apply(lambda x: ('00000'+str(x))[-11:])

In [18]:
data['OFFICE_PHONE'] = data['OFFICE_PHONE'].apply(lambda x: str(x).strip())

In [19]:
merge = data[['ME_NO', 'OFFICE_PHONE']].merge(dhc[['ME', 'Phone Number']], left_on='ME_NO', right_on='ME', how='left')
merge

,ME_NO,OFFICE_PHONE,ME,Phone Number
0,04204880064,7872818522,04204880064,7872818522
1,03545120088,6364417280,03545120088,6364417280
2,69007840063,4787466662,69007840063,4787466662
3,65501960025,8056526729,65501960025,nan
4,00802951248,4126411635,00802951248,4126411635
...,...,...,...,...
495,49529760051,5594551500,49529760051,nan
496,04177901658,7325367144,04177901658,7325367144
497,00801930336,3179639300,00801930336,3179630156
498,01803940262,5158759192,01803940262,5158759192


In [20]:
merge['match'] = merge['OFFICE_PHONE'] == merge['Phone Number']
merge['match'].sum()

277

### Coverage

In [24]:
1 - (merge['ME'].isna().sum() / len(merge))

0.948

### Match Rate

In [26]:
has_phone = merge[~merge['ME'].isna()]
len(has_phone)

474

In [27]:
has_phone['match'].sum() / len(has_phone)

0.5843881856540084

In [31]:
merge[['ME', 'OFFICE_PHONE', 'Phone Number']].rename(
    columns={'Phone Number': 'DHC_PHONE'}
).to_csv('dhc_comparison.csv', index=False)